In [1]:
import sys

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")

import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import tasks.run_helper
import tasks.preprocessing

In [4]:
def base_df():
    df = pd.read_json("../data/toxicity_ratings.json", lines=True)
    df = df.explode(column="ratings")

    ratings_df = pd.json_normalize(df.ratings)
    df = pd.concat([df.reset_index(), ratings_df.reset_index()], axis=1)
    df = df.drop(columns=["ratings", "index"])

    df = df.loc[
        :,
        [
            "comment",
            "toxic_score",
            "personally_seen_toxic_content",
            "personally_been_target",
            "identify_as_transgender",
            "toxic_comments_problem",
            "education",
            "age_range",
            "lgbtq_status",
            "political_affilation",
            "is_parent",
            "religion_important",
        ],
    ]
    df = df.groupby("comment").agg(list)
    df = df.head(5000)
    df = df.reset_index()
    return df

df = base_df()
df["random"] = tasks.preprocessing.get_rand_col(df, "education")

In [3]:
sdb_columns = [
    "personally_seen_toxic_content",
    "personally_been_target",
    "identify_as_transgender",
    "toxic_comments_problem",
    "education",
    "age_range",
    "lgbtq_status",
    "political_affilation",
    "is_parent",
    "religion_important",
]

res = tasks.run_helper.run_all_results(
    df=df,
    sdb_columns=sdb_columns,
    value_col="toxic_score",
    comment_key_col="comment",
)
res

Evaluating SDB dimensions:   0%|          | 0/10 [00:00<?, ?it/s]

In [1]:
tasks.run_helper.run_result(
    df,
    sdb_column="random",
    value_col="toxic_score",
    comment_key_col="comment",
)

NameError: name 'tasks' is not defined